In [ ]:
# 모듈화 시켜서 배포
# aws.py
# └ class Rekognition
#   └ compare_faces
#   └ detect_faces
#   └ detect_label
#   └ recognize_celeb

In [11]:
import boto3

def compare_faces(sourceFile, targetFile):

    client = boto3.client('rekognition')

    imageSource = open(sourceFile, 'rb')
    imageTarget = open(targetFile, 'rb')

    response = client.compare_faces(SimilarityThreshold=0,
                                    SourceImage={'Bytes': imageSource.read()},
                                    TargetImage={'Bytes': imageTarget.read()})
    result = ""
    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']
        
        # 사용자를 위한 UI
        result = f"동일 인물일 확률 : {faceMatch['Similarity'] :.2f}%"

    imageSource.close()
    imageTarget.close()
    if len(response['FaceMatches']) == 0:
        return "얼굴을 인식할 수 없습니다."
    else:
        return result
    return len(response['FaceMatches'])

def main():
    source_file = 'imgs/pbk2.jpeg'
    target_file = 'imgs/pbk.jpeg'
    face_matches = compare_faces(source_file, target_file)
    print(face_matches)
if __name__ == "__main__":
    main()

동일 인물일 확률 : 99.82%


In [18]:
import boto3
import json

def detect_faces(photo):
    
    client = boto3.client('rekognition')

    with open(photo, "rb") as image:
        response = client.detect_faces(Image={'Bytes' : image.read()}, Attributes=['ALL'])

    result = ""
    for faceDetail in response['FaceDetails']:

        # 성별 : 남성   (혹은 여성)
        gender = faceDetail['Gender']['Value']
        result += f"성별 : {'남성' if gender == 'Male' else '여성'}\n"
        # 나이 : 28(세) (24 ~ 32의 평균)
        age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) // 2
        result += f"나이 : {age}(세)\n"
        # 감정 : HAPPY
        emotion = faceDetail['Emotions'][0]['Type']
        result += f"감정 : {emotion}"

    if len(response['FaceDetails']) == 0:
        return "얼굴을 인식할 수 없습니다"
    else:
        return result
    
def main():
    photo='imgs/pbk2.jpeg'
    face_count=detect_faces(photo)
    print(face_count)

if __name__ == "__main__":
    main()

성별 : 남성
나이 : 17(세)
감정 : CALM


In [24]:
import boto3

def recognize_celebrities(photo):
    
    client = boto3.client('rekognition')

    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})

    result = ""
    # 닮은 연예인이 없는 경우 ... 누가 들어도 괜찮은 사람을 닮았다고 리턴
    celeb_list = ["송중기", "아리아나 그란데", "아이유", "장원영", "카리나", " 설윤", "송강", "강동원"]
    
    for celebrity in response['CelebrityFaces']:
        result = celebrity['Name']

    import random
    if len(response['CelebrityFaces']) == 0 :
        return random.choice(celeb_list)
        # 위 리스트 중 랜덤하게 1인 return 하기
        
    else:
        return result
        
def main():
    photo = 'imgs/img5.jpg'
    celeb_count = recognize_celebrities(photo)
    print("Celebrities detected: " + str(celeb_count))

if __name__ == "__main__":
    main()

Celebrities detected:  설윤


In [32]:
import boto3

# 1) 이미지를 던졌을 때, 해당 label을 return해주는 서비스

def detect_labels_local_file(photo):


    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
    result = ""   
    for label in response['Labels']:
        result += f"{label['Name']} : {label['Confidence']:.2f}%\n"
    if len(response['Labels']) == 0 :
        return "잘못된 이미지 입니다."
    else:
        return result
    return len(response['Labels'])

def main():
    photo='imgs/pengsoo.png'

    label_count=detect_labels_local_file(photo)
    print(label_count)


if __name__ == "__main__":
    main()

Nature : 93.37%
Outdoors : 93.37%
Snow : 93.37%
Snowman : 93.37%
Winter : 93.37%
Plush : 89.48%
Toy : 89.48%
Cartoon : 61.97%
Bathroom : 55.14%
Indoors : 55.14%
Room : 55.14%
Toilet : 55.14%



In [58]:
import boto3

# 2) 이 이미지가 강아지인지 아닌지 확인하고 싶어서
#   -> 강아지 일 확률은 95%입니다
#   -> "강아지가 아닙니다"

def detect_is_label(photo, key):
                                # 확인하고자 하는 key 강아지? 고양이 ?

    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
    result = "" 
    # 강아지는 Dog 라는 key를 가지고 있다.
    # 해당 key가 존재한다면
    # 사용자에게 " 강아지일 확률은 95.05입니다. "return

    is_checked = False
    # 사용자에게 " 강아지가 아닙니다. " return
    
    for label in response['Labels']:
        if label['Name'] == key:
            result += f"{key} 일 확률은 : {label['Confidence']:.2f}% 입니다.\n"
            is_checked = True
            break
            
    if len(response['Labels']) == 0 :
        return "잘못된 이미지 입니다."
    elif is_checked == True:
        return result
    else:
        return f"{key}이(가) 아닙니다."

def main():
    photo='imgs/cat.jpeg'

    label_count = detect_is_label(photo,"Dog")
    print(label_count)


if __name__ == "__main__":
    main()

Dog이(가) 아닙니다.


In [4]:
class Rekognition:
    def compare_faces(self, sourceFile, targetFile):

        client = boto3.client('rekognition')
    
        imageSource = open(sourceFile, 'rb')
        imageTarget = open(targetFile, 'rb')
    
        response = client.compare_faces(SimilarityThreshold=0,
                                        SourceImage={'Bytes': imageSource.read()},
                                        TargetImage={'Bytes': imageTarget.read()})
    
        result = ""
        for faceMatch in response['FaceMatches']:
            position = faceMatch['Face']['BoundingBox']
    
            result = f"동일 인물일 확률 :{faceMatch['Similarity'] : .2f}%"
        imageSource.close()
        imageTarget.close()
        if len(response['FaceMatches']) == 0:
            return "얼굴을 인식할 수 없습니다"
        else:
            return result

    def detect_faces(self, photo):
        
        client = boto3.client('rekognition')
    
        with open(photo, "rb") as image:
            response = client.detect_faces(Image={'Bytes' : image.read()}, Attributes=['ALL'])
    
        result = ""
        for faceDetail in response['FaceDetails']:
    
            # 성별 : 남성   (혹은 여성)
            gender = faceDetail['Gender']['Value']
            result += f"성별 : {'남성' if gender == 'Male' else '여성'}\n"
            # 나이 : 28(세) (24 ~ 32의 평균)
            age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) // 2
            result += f"나이 : {age}(세)\n"
            # 감정 : HAPPY
            emotion = faceDetail['Emotions'][0]['Type']
            result += f"감정 : {emotion}"
    
        if len(response['FaceDetails']) == 0:
            return "얼굴을 인식할 수 없습니다"
        else:
            return result
    
    def recognize_celebrities(self, photo):
        
        client = boto3.client('rekognition')
    
        with open(photo, 'rb') as image:
            response = client.recognize_celebrities(Image={'Bytes': image.read()})
    
        result = ""
        # 닮은 연예인이 없는 경우...누가 들어도 괜찮은 사람을 닮았다고 리턴
        celeb_list = ["송중기", "아라아나 그란데", "아이유", "장원영", "카리나", "설윤", "송강", "강동원"]
        
        for celebrity in response['CelebrityFaces']:
            result = celebrity['Name']
    
        import random
        
        if len(response['CelebrityFaces']) == 0:
            return random.choice(celeb_list)
        else:
            return result
    
    def detect_labels_local_file(self, photo):
    
        client=boto3.client('rekognition')
       
        with open(photo, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
            
        result = ""
        for label in response['Labels']:
            result += f"{label['Name']} : {label['Confidence']:.2f}%\n"
    
        if len(response['Labels']) == 0:
            return "잘못된 이미지입니다"
        else:
            return result
    
    def detect_is_label(self, photo, key):
    
        client=boto3.client('rekognition')
       
        with open(photo, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
            
        result = ""
    
        is_checked = False
        
        for label in response['Labels']:
            if label['Name'] == key:
                result += f"{key}일 확률은 {label['Confidence']:.2f}%입니다\n"
                is_checked = True
                break
    
        if len(response['Labels']) == 0:
            return "잘못된 이미지입니다"
        elif is_checked == True:
            return result
        else:
            return f"{key}이(가) 아닙니다"

In [2]:
# 이 사진이 고양이 사진인지 판별
from aws import detect_is_label
result = detect_is_label("imgs/cat.jpeg","Cat")
print(result)

Cat일 확률은 99.70%입니다



In [ ]:
# API KEY
# 특정 서버에 GET방식으로 Client가 요청 (request) 
# Server가 응답 (response)

In [6]:
 # 127.0.0.1은 IPv4 주소이다
    # IP란 Server가 어디에 있는지 알려주는 주소
    # 네이버 서버 -> 25.50.127.255 라면 이대로 치면 네이버로 연결
    # 영문명(도메인) -> IPv4 변환
    # 나중에 서버를 운영하려면, 서버IP구매 + 도메인도 구매
    # 127.0.0.1 (Localhost) -> "나"를 가르킴
    # 누군가가 나를 바라보게 하고싶다 ? 공개 IP가 필요
    # 공개 IPv4는 2의32승 -> 42~3억개만 전 세계에 존재

    # 개인(Private) IPv4를 통해 다른사람 서버를 바라보자
    # 같은 인터넷망(공유기)에서는 개인 IPv4를 바라볼 수 있음
    

# 파이썬에서 구성되는 웹  프레임 워크 : Flask , Django
# 1) Flask : 마이크로 웹 프레임워크
from flask import Flask # 서버를 라우팅, 구동하는 클래스
app = Flask(__name__)
@app.route("/")
def index():
    return "<h1>이현의 웹 페이지</h1>"
if __name__ =="__main__":
    #host= "0.0.0.0": 누구나 내 서버에 접속 가능
    app.run(host="0.0.0.0")

#IPv4/8bit(0~255)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.10.141.4:5000
Press CTRL+C to quit
10.10.141.249 - - [17/Sep/2025 10:57:38] "GET / HTTP/1.1" 200 -
10.10.141.4 - - [17/Sep/2025 11:48:02] "GET / HTTP/1.1" 200 -
10.10.141.4 - - [17/Sep/2025 11:48:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Sep/2025 11:48:05] "GET / HTTP/1.1" 200 -


In [7]:
!pip list | findstr "flask"

In [ ]:
! pip install flask

In [8]:
# window powershell
# sudo passwd root 1234
# sudo apt-get update
# sudo apt-get upgrade
# sudo apt-get python3.9
# sudo apt-get python3-pip
# pip install flask